# Multi-Armed Bandits

Do you have a favourite coffee place in town? When you feel the need to replenish your caffeine levels, chances are you will choose to get a coffee from your favourite coffee place, as you are almost sure that you will get the best coffee or at least closer to your subjective preferences. This however means that you may be missing out on the delicious coffee served by this new place two blocks away. And if you were to try out all the coffee places one by one, the probability of having the worse coffee of your life would be pretty high! But then again, there’s a chance you’ll find an even better coffee brewer. 

The dilemma in our coffee tasting experiment arises from incomplete information. Put it differently, it is important to gather enough information in order to formulate the best overall strategy and then explore new actions. This will eventually lead to minimizing the overall bad experiences.

A multi-armed bandit is a simplified form of this analogy. It is used to represent similar kinds of problems and finding a good strategy to solve them is already helping a lot of industries. In this Python notebook, we will explore some strategies on how to solve the multi armed bandit problem.

![robot](img/robot.png)

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

## 1. The general bandit algorithm

Here we declare a general bandit algorithm that accepts various initialization, action selection and update strategies.

In [ ]:
def bandit(bandit_inputs, seed=1):
    
    # Unpacking the bandit inputs:
    k = bandit_inputs["k"]
    N = bandit_inputs["N"]
    q_star = bandit_inputs["q_star"]
    initialization = bandit_inputs["initialization"]
    action_selection = bandit_inputs["action_selection"]
    update_rule = bandit_inputs["update_rule"]
    
    #Initializing rewards and regret
    total_reward = 0
    mean_reward = 0
    mean_rewards = np.zeros(N)
    regret = np.zeros(N)
    
    # Initialize the estimates
    np.random.seed(seed)
    estimates = initialization(k, q_star)
    for t in range(1,N+1):
        np.random.seed(seed*t) #we need this because random init changes the order later on
        # Select a coffee shop
        coffee_shop = action_selection(estimates, bandit_inputs, t)
    
        # Update action counter
        bandit_inputs["k_t"][coffee_shop] += 1
    
        # Get reward
        reward = np.random.normal(q_star[coffee_shop], 1)
        
        # Update the total reward
        total_reward += reward
        
        # Calculate mean reward and regret
        mean_reward = mean_reward + (reward - mean_reward) / t
        mean_rewards[t-1] = mean_reward
        #regret[t-1] = t * mean_rewards[np.argmax(mean_rewards[0:t])] - total_reward
        regret[t-1] = np.max(q_star) - reward # True maximum! We do not know it but, why not? :)
        #if regret[t-1]<0:
            #print("what?")
        
        estimates = update_rule(estimates, coffee_shop, reward, bandit_inputs, mean_reward)
        
    return estimates, total_reward, mean_rewards, regret

## 1.1. Initialization strategies
Using the q* values, we create initialization methods that implement zero, pessimistic, average, optimistic and random estimate initializations for the bandit algorithm:

#### Estimate initialization with zeroes:

In [ ]:
def ZeroInitialization(k, q_star):
    # Zero initialization
    return np.zeros(k)

#### Pessimistic estimate initialization:

In [ ]:
def MinInitialization(k, q_star):
    # Min initialization
    return np.ones(k) * np.min(q_star)

#### Mean estimate initialization:

In [ ]:
def MeanInitialization(k, q_star):
    # Mean initialization
    return np.ones(k) * np.mean(q_star)

#### Optimistic estimate initialization:

In [ ]:
def MaxInitialization(k, q_star):
    # Max initialization
    return np.ones(k) * np.max(q_star)

#### Estimate initialization with random numbers:

In [ ]:
def RandomInitialization(k, q_star):
    # Random initialization
    return np.random.normal(0, 1, k)  #let's assume we know the distributions and take nice random initial values :)

## 1.2. Action selection strategies

#### Random action selection: 
This strategy selects an action completely at random without taking into account the learned estimates.

In [ ]:
def random_action_selection(estimates, bandit_inputs, t):
    # Choose randomly (uniform)
    return np.random.randint(bandit_inputs["k"])

#### Greedy action selection:
This action selection strategy always selects the best possible action.

In [ ]:
def greedy_action_selection(estimates, bandit_inputs, t):
    # Choose randomly
    return np.argmax(estimates)

#### ε-Greedy action selection:
The implementation of the e-Greedy algorithm for the multi-armed bandit problem.

In [ ]:
def e_greedy_action_selection(estimates, bandit_inputs, t):
    # Generate a random number
    p = np.random.rand()
    
    # E-Greedy action selection
    if p < bandit_inputs["epsilon"]:
        # Randomly select an action
        return np.random.choice(bandit_inputs["k"])
    else:
        # Take greedy action
        return np.argmax(estimates)

#### ε-Greedy with epsion decay action selection:
The implementation of the e-Greedy algorithm with epsilon decay for the multi-armed bandit problem.

In [ ]:
def e_decay_greedy_action_selection(estimates, bandit_inputs, t):

    # Generate a random number
    p = np.random.rand()

    # E-Greedy action selection
    if p < bandit_inputs["epsilon"]*np.exp(-bandit_inputs["kappa"]*t):
        # Randomly select an action
        action = np.random.choice(bandit_inputs["k"])
    else:
        # Take greedy action
        action = np.argmax(estimates)

    #if bandit_inputs["epsilon"] > bandit_inputs["min_epsilon"]:
        #bandit_inputs["epsilon"] = bandit_inputs["e0"]*np.exp(-bandit_inputs["kappa"]*(np.sum(bandit_inputs["k_t"])+1))
        #if bandit_inputs["epsilon"] < bandit_inputs["min_epsilon"]:
            #bandit_inputs["epsilon"] = bandit_inputs["min_epsilon"]
            
    return action

#### Upper Confidence Bound action selection:
The implementation of the UCB algorithm for the multi armed bandit problem.

In [ ]:
def ucb_action_selection(estimates, bandit_inputs, t):
    t = np.sum(bandit_inputs["k_t"]) + 1
    # Select action according to UCB Criteria
    return np.argmax(estimates + bandit_inputs["c"] * np.sqrt(np.log(t) / (bandit_inputs["k_t"]+1)))

#### SoftMax action selection:
The implementation of the softmax bandit algorithm for the multi-armed bandit problem. 

In [ ]:
def softmax_action_selection(estimates, bandit_inputs, t):
    # Softmax action selection
    action = np.random.choice(bandit_inputs["k"], 1, \
                              p=np.exp(estimates/bandit_inputs["tau"]) / np.sum(np.exp(estimates/bandit_inputs["tau"])))
    return action[0]

## 1.3. Update strategies 

#### True average update rule:

In [ ]:
def update_rule(estimates, coffee_shop, reward, bandit_inputs, mean_reward=None):
    # Update the estimates
    estimates[coffee_shop] = estimates[coffee_shop] + \
                            (reward - estimates[coffee_shop]) / bandit_inputs["k_t"][coffee_shop]
    return estimates

#### Constant learning rate update rule:

In [ ]:
def update_rule_constant_lr(estimates, coffee_shop, reward, bandit_inputs, mean_reward=None):
    # Update the estimates
    estimates[coffee_shop] = estimates[coffee_shop] + \
                            (reward - estimates[coffee_shop]) / (bandit_inputs["k_t"][coffee_shop]*bandit_inputs["a"])
    return estimates

What about decaying learning rate?

# 2. The Experiment

Here we declare the number of trials and arms of the multi-armed bandit problem. We also select a gaussian probability distribution to generate the true values of each arm and to simulate the randomness of the experiment. <br>After each experiment we print two plots to evaluate the performance of the algorithm. We print the loss that we incur due to time/rounds spent due to the learning, or else the regret, and the expected reward of the algorithm across the rounds of each experiment.

In [ ]:
# Number of coffee shops
k = 10

# Number of trials
N = 10000

# Declaration of the true q* values according to a gaussian distribution
q_star = np.random.normal(0, 1, k)
print(q_star)

#### Plotting/printing:
Definition of the print and plot of the results (We use this across the whole notebook):

In [ ]:
def plot_bandits(N, estimates, total_reward, mean_rewards, regret):
    print("Learned Estimates: {}".format(estimates))
    print("")
    print("Euclidean distance from q_star vector: {}".format(np.linalg.norm(q_star-estimates)))
    print("Total Reward: {}".format(total_reward))
    print("Mean Reward: {}".format(mean_rewards[-1]))

    fig, ax = plt.subplots()
    ax.plot(np.linspace(0,N-1,N), np.cumsum(regret), 'b-')
    ax.set_xlabel("Time")
    ax.set_ylabel("Speed")
    ax.grid()
    ax.set(xlabel='Time Steps', ylabel='Total Regret',
           title='Regret Curve')

    plt.show()

    fig, ax = plt.subplots()
    ax.plot(np.linspace(0,N-1,N), mean_rewards, 'b-')
    ax.set_xlabel("Time")
    ax.set_ylabel("Speed")
    ax.grid()
    ax.set(xlabel='Time Steps', ylabel='Average Reward',
           title='Average Reward Curve')

    plt.show()

Definition of the comp_plot, that plots the comparative plot of the various initializations:

In [ ]:
def comp_plot(regret, pes_regret, avg_regret, opt_regret, rnd_regret, \
              mean_rewards, pes_mean_rewards, avg_mean_rewards, opt_mean_rewards, rnd_mean_rewards):
    fig, ax = plt.subplots()
    ax.plot(np.linspace(0,N-1,N), np.cumsum(regret), 'b-')
    ax.plot(np.linspace(0,N-1,N), np.cumsum(opt_regret), 'g-')
    ax.plot(np.linspace(0,N-1,N), np.cumsum(avg_regret), 'r-')
    ax.plot(np.linspace(0,N-1,N), np.cumsum(pes_regret), 'c-')
    ax.plot(np.linspace(0,N-1,N), np.cumsum(rnd_regret), 'y-')
    ax.set_xlabel("Time")
    ax.set_ylabel("Speed")
    ax.grid()
    ax.set(xlabel='Time Steps', ylabel='Total Regret',
           title='Regret Curve')
    plt.legend(['Zero','Optimistic','Mean','Pesimistic','Random'])
    plt.show()

    fig, ax = plt.subplots()
    ax.plot(np.linspace(0,N-1,N), mean_rewards, 'b-')
    ax.plot(np.linspace(0,N-1,N), opt_mean_rewards, 'g-')
    ax.plot(np.linspace(0,N-1,N), avg_mean_rewards, 'r-')
    ax.plot(np.linspace(0,N-1,N), pes_mean_rewards, 'c-')
    ax.plot(np.linspace(0,N-1,N), rnd_mean_rewards, 'y-')
    ax.set_xlabel("Time")
    ax.set_ylabel("Speed")
    ax.grid()
    ax.set(xlabel='Time Steps', ylabel='Average Reward',
           title='Average Reward Curve')
    plt.legend(['Zero','Optimistic','Mean','Pesimistic','Random'])
    plt.show()

## 2.1. The bandit algorithm with random action selection and true average update

#### Initializing with zero values:
What do you think the expected reward is? 

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "initialization"    : ZeroInitialization, # Initialization strategy
                 "action_selection"  : random_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }
estimates_rand, total_reward_rand, mean_rewards_rand, regret_rand = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, estimates_rand, total_reward_rand, mean_rewards_rand, regret_rand)

#### Initializing with Pessimistic:
What do you think the expected reward is? 

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "initialization"    : MinInitialization, # Initialization strategy
                 "action_selection"  : random_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }
min_estimates_rand, min_total_reward_rand, min_mean_rewards_rand, min_regret_rand = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, min_estimates_rand, min_total_reward_rand, min_mean_rewards_rand, min_regret_rand)

#### Initializing with Mean:
What do you think the expected reward is?

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "initialization"    : MeanInitialization, # Initialization strategy
                 "action_selection"  : random_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

avg_estimates_rand, avg_total_reward_rand, avg_mean_rewards_rand, avg_regret_rand = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, avg_estimates_rand, avg_total_reward_rand, avg_mean_rewards_rand, avg_regret_rand)

#### Initializing with Optimistic:
What do you think the expected reward is?

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "initialization"    : MaxInitialization, # Initialization strategy
                 "action_selection"  : random_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }
max_estimates_rand, max_total_reward_rand, max_mean_rewards_rand, max_regret_rand = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, max_estimates_rand, max_total_reward_rand, max_mean_rewards_rand, max_regret_rand)

#### Initializing with Random
What do you think the expected reward is?

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "initialization"    : RandomInitialization, # Initialization strategy
                 "action_selection"  : random_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

rnd_estimates_rand, rnd_total_reward_rand, rnd_mean_rewards_rand, rnd_regret_rand = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, rnd_estimates_rand, rnd_total_reward_rand, rnd_mean_rewards_rand, rnd_regret_rand)

#### Compare all:

Let's see a comparison between the different initialization strategies for the random action selection strategy:

In [ ]:
comp_plot(regret_rand, min_regret_rand, avg_regret_rand, max_regret_rand, rnd_regret_rand, \
          mean_rewards_rand, min_mean_rewards_rand, avg_mean_rewards_rand, max_mean_rewards_rand, rnd_mean_rewards_rand)

## 2.2. The bandit algorithm with greedy action selection and true average update

#### Initializing with zero values:
What do you think the expected reward is? 

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "initialization"    : ZeroInitialization, # Initialization strategy
                 "action_selection"  : greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

estimates_greedy, total_reward_greedy, mean_rewards_greedy, regret_greedy = bandit(bandit_inputs)
print("True q values:{}".format(q_star))                 
plot_bandits(N, estimates_greedy, total_reward_greedy, mean_rewards_greedy, regret_greedy)

#### Initializing with Pessimistic:
What do you think the expected reward is? 

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "initialization"    : MinInitialization, # Initialization strategy
                 "action_selection"  : greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

min_estimates_greedy, min_total_reward_greedy, min_mean_rewards_greedy, min_regret_greedy = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, min_estimates_greedy, min_total_reward_greedy, min_mean_rewards_greedy, min_regret_greedy)

#### Initializing with Mean:
What do you think the expected reward is?

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "initialization"    : MeanInitialization, # Initialization strategy
                 "action_selection"  : greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

avg_estimates_greedy, avg_total_reward_greedy, avg_mean_rewards_greedy, avg_regret_greedy = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, avg_estimates_greedy, avg_total_reward_greedy, avg_mean_rewards_greedy, avg_regret_greedy)

#### Initializing with Optimistic:
What do you think the expected reward is?

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "initialization"    : MaxInitialization, # Initialization strategy
                 "action_selection"  : greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

max_estimates_greedy, max_total_reward_greedy, max_mean_rewards_greedy, max_regret_greedy = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, max_estimates_greedy, max_total_reward_greedy, max_mean_rewards_greedy, max_regret_greedy)

#### Initializing with Random
What do you think the expected reward is?
(we will use the average of 100 random initializations, so that the results are significant):

In [ ]:
trials = 100

rnd_estimates_greedy = np.zeros(k)
rnd_total_reward_greedy = 0
rnd_mean_rewards_greedy = np.zeros(N)
rnd_regret_greedy = np.zeros(N)

for i in range(trials):
    bandit_inputs = {"k"             : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "initialization"    : RandomInitialization, # Initialization strategy
                 "action_selection"  : greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

    i_estimates_greedy, i_total_reward_greedy, i_mean_rewards_greedy, i_regret_greedy = bandit(bandit_inputs, i)
    rnd_estimates_greedy += i_estimates_greedy
    rnd_total_reward_greedy += i_total_reward_greedy
    rnd_mean_rewards_greedy += i_mean_rewards_greedy
    rnd_regret_greedy += i_regret_greedy
    #print(rnd_regret_greedy)

rnd_estimates_greedy /= trials
rnd_total_reward_greedy /= trials
rnd_mean_rewards_greedy /= trials
rnd_regret_greedy /= trials
print("True q values:{}".format(q_star))
plot_bandits(N, rnd_estimates_greedy, rnd_total_reward_greedy, rnd_mean_rewards_greedy, rnd_regret_greedy)

#### Compare all:

Let's see a comparison between the different initialization strategies for the random action selection strategy:

In [ ]:
comp_plot(regret_greedy, min_regret_greedy, avg_regret_greedy, max_regret_greedy, rnd_regret_greedy, \
          mean_rewards_greedy, min_mean_rewards_greedy, avg_mean_rewards_greedy, max_mean_rewards_greedy, \
          rnd_mean_rewards_greedy)

## e-Greedy action selection strategy

### Test out different epsilon values. (Especially the limit cases 0, 1)

#### Initializing with zero values:
What do you think the expected reward is? 

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "epsilon"           : 0.1, # Epsilon parameter 
                 "initialization"    : ZeroInitialization, # Initialization strategy
                 "action_selection"  : e_greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

estimates_e_greedy, total_reward_e_greedy, mean_rewards_e_greedy, regret_e_greedy = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, estimates_e_greedy, total_reward_e_greedy, mean_rewards_e_greedy, regret_e_greedy)

#### Initializing with Pessimistic:
What do you think the expected reward is? 

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "epsilon"           : 0.1,
                 "initialization"    : MinInitialization, # Initialization strategy
                 "action_selection"  : e_greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

min_estimates_e_greedy, min_total_reward_e_greedy, min_mean_rewards_e_greedy, min_regret_e_greedy = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, min_estimates_e_greedy, min_total_reward_e_greedy, min_mean_rewards_e_greedy, min_regret_e_greedy)

#### Initializing with Mean:
What do you think the expected reward is?

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "epsilon"           : 0.1,
                 "initialization"    : MeanInitialization, # Initialization strategy
                 "action_selection"  : e_greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

avg_estimates_e_greedy, avg_total_reward_e_greedy, avg_mean_rewards_e_greedy, avg_regret_e_greedy = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, avg_estimates_e_greedy, avg_total_reward_e_greedy, avg_mean_rewards_e_greedy, avg_regret_e_greedy)

#### Initializing with Optimistic:
What do you think the expected reward is?

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "epsilon"           : 0.1,
                 "initialization"    : MaxInitialization, # Initialization strategy
                 "action_selection"  : e_greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

max_estimates_e_greedy, max_total_reward_e_greedy, max_mean_rewards_e_greedy, max_regret_e_greedy = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, max_estimates_e_greedy, max_total_reward_e_greedy, max_mean_rewards_e_greedy, max_regret_e_greedy)

#### Initializing with Random
What do you think the expected reward is?
(we will use the average of 100 random initializations, so that the results are significant):

In [ ]:
trials = 100

rnd_estimates_e_greedy = np.zeros(k)
rnd_total_reward_e_greedy = 0
rnd_mean_rewards_e_greedy = np.zeros(N)
rnd_regret_e_greedy = np.zeros(N)

for i in range(trials):
    bandit_inputs = {"k"             : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "epsilon"           : 0.1,
                 "initialization"    : RandomInitialization, # Initialization strategy
                 "action_selection"  : e_greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }
    i_estimates_e_greedy, i_total_reward_e_greedy, i_mean_rewards_e_greedy, i_regret_e_greedy = bandit(bandit_inputs)
    rnd_estimates_e_greedy += i_estimates_e_greedy
    rnd_total_reward_e_greedy += i_total_reward_e_greedy
    rnd_mean_rewards_e_greedy += i_mean_rewards_e_greedy
    rnd_regret_e_greedy += i_regret_e_greedy

rnd_estimates_e_greedy /= trials
rnd_total_reward_e_greedy /= trials
rnd_mean_rewards_e_greedy /= trials
rnd_regret_e_greedy /= trials
print("True q values:{}".format(q_star))
plot_bandits(N, rnd_estimates_e_greedy, rnd_total_reward_e_greedy, rnd_mean_rewards_e_greedy, rnd_regret_e_greedy)

#### Compare all:

Let's see a comparison between the different initialization strategies for the e-greedy action selection strategy:

In [ ]:
comp_plot(regret_e_greedy, min_regret_e_greedy, avg_regret_e_greedy, max_regret_e_greedy, rnd_regret_e_greedy, \
          mean_rewards_e_greedy, min_mean_rewards_e_greedy, avg_mean_rewards_e_greedy, max_mean_rewards_e_greedy, \
          rnd_mean_rewards_e_greedy)

## e-Greedy with epsilon decay action selection strategy

#### Initializing with zero values:
What do you think the expected reward is? 

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "kappa"             : 0.001, # Decay coefficient
                 "epsilon"           : 0.1,  # The epsilon parameter. (try with 0.01 as well--will it get stuck to lockal minima?)
                 "initialization"    : ZeroInitialization, # Initialization strategy
                 "action_selection"  : e_decay_greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

estimates_e_decay_greedy, total_reward_e_decay_greedy, mean_rewards_e_decay_greedy, regret_e_decay_greedy = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, estimates_e_decay_greedy, total_reward_e_decay_greedy, mean_rewards_e_decay_greedy, regret_e_decay_greedy)

### Test out different epsilon, min_epsilon and kappa values. (Especially the limit cases 0, 1)

#### Initializing with Pessimistic:
What do you think the expected reward is? 

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "kappa"             : 0.001, # Decay coefficient
                 "epsilon"           : 0.1,  # The epsilon parameter.
                 "initialization"    : MinInitialization, # Initialization strategy
                 "action_selection"  : e_decay_greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

min_estimates_e_decay_greedy, min_total_reward_e_decay_greedy, min_mean_rewards_e_decay_greedy, min_regret_e_decay_greedy = bandit(bandit_inputs)
print("True q values:{}".format(q_star))                 
plot_bandits(N, min_estimates_e_decay_greedy, min_total_reward_e_decay_greedy, min_mean_rewards_e_decay_greedy, min_regret_e_decay_greedy)

#### Initializing with Mean:
What do you think the expected reward is?

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "kappa"             : 0.001, # Decay coefficient
                 "epsilon"           : 0.1,  # The epsilon parameter.
                 "initialization"    : MeanInitialization, # Initialization strategy
                 "action_selection"  : e_decay_greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

avg_estimates_e_decay_greedy, avg_total_reward_e_decay_greedy, avg_mean_rewards_e_decay_greedy, avg_regret_e_decay_greedy = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, avg_estimates_e_decay_greedy, avg_total_reward_e_decay_greedy, avg_mean_rewards_e_decay_greedy, avg_regret_e_decay_greedy)

#### Initializing with Optimistic:
What do you think the expected reward is?

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "kappa"             : 0.001, # Decay coefficient
                 "epsilon"           : 0.1,  # The epsilon parameter.
                 "initialization"    : MaxInitialization, # Initialization strategy
                 "action_selection"  : e_decay_greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

max_estimates_e_decay_greedy, max_total_reward_e_decay_greedy, max_mean_rewards_e_decay_greedy, max_regret_e_decay_greedy = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, max_estimates_e_decay_greedy, max_total_reward_e_decay_greedy, max_mean_rewards_e_decay_greedy, max_regret_e_decay_greedy)

#### Initializing with Random
What do you think the expected reward is?
(we will use the average of 100 random initializations, so that the results are significant):

In [ ]:
trials = 100

rnd_estimates_e_decay_greedy = np.zeros(k)
rnd_total_reward_e_decay_greedy = 0
rnd_mean_rewards_e_decay_greedy = np.zeros(N)
rnd_regret_e_decay_greedy = np.zeros(N)

for i in range(trials):
    bandit_inputs = {"k"             : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "kappa"             : 0.001, # Decay coefficient
                 "epsilon"           : 0.1,  # The epsilon parameter.
                 "initialization"    : RandomInitialization, # Initialization strategy
                 "action_selection"  : e_decay_greedy_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }
    i_estimates_e_decay_greedy, i_total_reward_e_decay_greedy, i_mean_rewards_e_decay_greedy, i_regret_e_decay_greedy = bandit(bandit_inputs)
    rnd_estimates_e_decay_greedy += i_estimates_e_decay_greedy
    rnd_total_reward_e_decay_greedy += i_total_reward_e_decay_greedy
    rnd_mean_rewards_e_decay_greedy += i_mean_rewards_e_decay_greedy
    rnd_regret_e_decay_greedy += i_regret_e_decay_greedy

rnd_estimates_e_decay_greedy /= trials
rnd_total_reward_e_decay_greedy /= trials
rnd_mean_rewards_e_decay_greedy /= trials
rnd_regret_e_decay_greedy /= trials
print("True q values:{}".format(q_star))
plot_bandits(N, rnd_estimates_e_decay_greedy, rnd_total_reward_e_decay_greedy, rnd_mean_rewards_e_decay_greedy, rnd_regret_e_decay_greedy)

#### Compare all:

Let's see a comparison between the different initialization strategies for the e-Greedy with epsilon decay action selection strategy:

In [ ]:
comp_plot(regret_e_decay_greedy, min_regret_e_decay_greedy, avg_regret_e_decay_greedy,\
          max_regret_e_decay_greedy, rnd_regret_e_decay_greedy, mean_rewards_e_decay_greedy,\
          min_mean_rewards_e_decay_greedy, avg_mean_rewards_e_decay_greedy,\
          max_mean_rewards_e_decay_greedy, rnd_mean_rewards_e_decay_greedy)

## Upper Confidence Bound action selection strategy

### Test out different c values.

#### Initializing with zero values:
What do you think the expected reward is? 

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "c"                 : 2., # The c parameter
                 "initialization"    : ZeroInitialization, # Initialization strategy
                 "action_selection"  : ucb_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

estimates_ucb, total_reward_ucb, mean_rewards_ucb, regret_ucb = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, estimates_ucb, total_reward_ucb, mean_rewards_ucb, regret_ucb)

#### Initializing with Pessimistic:
What do you think the expected reward is? 

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "c"                 : 2., # The c parameter
                 "initialization"    : MinInitialization, # Initialization strategy
                 "action_selection"  : ucb_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

min_estimates_ucb, min_total_reward_ucb, min_mean_rewards_ucb, min_regret_ucb = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, min_estimates_ucb, min_total_reward_ucb, min_mean_rewards_ucb, min_regret_ucb)

#### Initializing with Mean:
What do you think the expected reward is?

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "c"                 : 2., # The c parameter
                 "initialization"    : MeanInitialization, # Initialization strategy
                 "action_selection"  : ucb_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

avg_estimates_ucb, avg_total_reward_ucb, avg_mean_rewards_ucb, avg_regret_ucb = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, avg_estimates_ucb, avg_total_reward_ucb, avg_mean_rewards_ucb, avg_regret_ucb)

#### Initializing with Optimistic:
What do you think the expected reward is?

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "c"                 : 2., # The c parameter
                 "initialization"    : MaxInitialization, # Initialization strategy
                 "action_selection"  : ucb_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

max_estimates_ucb, max_total_reward_ucb, max_mean_rewards_ucb, max_regret_ucb = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, max_estimates_ucb, max_total_reward_ucb, max_mean_rewards_ucb, max_regret_ucb)

#### Initializing with Random
What do you think the expected reward is?
(we will use the average of 100 random initializations, so that the results are significant):

In [ ]:
trials = 100

rnd_estimates_ucb = np.zeros(k)
rnd_total_reward_ucb = 0
rnd_mean_rewards_ucb = np.zeros(N)
rnd_regret_ucb = np.zeros(N)

for i in range(trials):
    bandit_inputs = {"k"             : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "c"                 : 2., # The c parameter
                 "initialization"    : RandomInitialization, # Initialization strategy
                 "action_selection"  : ucb_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }
    i_estimates_ucb, i_total_reward_ucb, i_mean_rewards_ucb, i_regret_ucb = bandit(bandit_inputs)
    rnd_estimates_ucb += i_estimates_ucb
    rnd_total_reward_ucb += i_total_reward_ucb
    rnd_mean_rewards_ucb += i_mean_rewards_ucb
    rnd_regret_ucb += i_regret_ucb

rnd_estimates_ucb /= trials
rnd_total_reward_ucb /= trials
rnd_mean_rewards_ucb /= trials
rnd_regret_ucb /= trials
print("True q values:{}".format(q_star))
plot_bandits(N, rnd_estimates_ucb, rnd_total_reward_ucb, rnd_mean_rewards_ucb, rnd_regret_ucb)

#### Compare all:

Let's see a comparison between the different initialization strategies for the ucb action selection strategy:

In [ ]:
comp_plot(regret_ucb, min_regret_ucb, avg_regret_ucb, max_regret_ucb, rnd_regret_ucb, \
          mean_rewards_ucb, min_mean_rewards_ucb, avg_mean_rewards_ucb, max_mean_rewards_ucb, \
          rnd_mean_rewards_ucb)

# Softmax action selection strategy

### Test out different tau values. What happens very close to 0 and with very high tau values? Try to find a good tau parameter!

#### Initializing with zero values:
What do you think the expected reward is? 

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "tau"               : 0.5, # The tau parameter
                 "initialization"    : ZeroInitialization, # Initialization strategy
                 "action_selection"  : softmax_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

estimates_sm, total_reward_sm, mean_rewards_sm, regret_sm = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, estimates_sm, total_reward_sm, mean_rewards_sm, regret_sm)

#### Initializing with Pessimistic:
What do you think the expected reward is? 

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "tau"               : 0.5, # The tau parameter
                 "initialization"    : MinInitialization, # Initialization strategy
                 "action_selection"  : softmax_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

min_estimates_sm, min_total_reward_sm, min_mean_rewards_sm, min_regret_sm = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, min_estimates_sm, min_total_reward_sm, min_mean_rewards_sm, min_regret_sm)

#### Initializing with Mean:
What do you think the expected reward is?

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "tau"               : 0.5, # The tau parameter
                 "initialization"    : MeanInitialization, # Initialization strategy
                 "action_selection"  : softmax_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

avg_estimates_sm, avg_total_reward_sm, avg_mean_rewards_sm, avg_regret_sm = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, avg_estimates_sm, avg_total_reward_sm, avg_mean_rewards_sm, avg_regret_sm)

#### Initializing with Optimistic:
What do you think the expected reward is?

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "tau"               : 0.5, # The tau parameter
                 "initialization"    : MaxInitialization, # Initialization strategy
                 "action_selection"  : softmax_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }

max_estimates_sm, max_total_reward_sm, max_mean_rewards_sm, max_regret_sm = bandit(bandit_inputs)
print("True q values:{}".format(q_star))
plot_bandits(N, max_estimates_sm, max_total_reward_sm, max_mean_rewards_sm, max_regret_sm)

#### Initializing with Random
What do you think the expected reward is?
(we will use the average of 100 random initializations, so that the results are significant):

In [ ]:
trials = 100

rnd_estimates_sm = np.zeros(k)
rnd_total_reward_sm = 0
rnd_mean_rewards_sm = np.zeros(N)
rnd_regret_sm = np.zeros(N)

for i in range(trials):
    bandit_inputs = {"k"             : k, # Number of coffee shops
                 "k_t"               : np.zeros(k), # The coffee shop selection counter
                 "N"                 : N, # Number of trials
                 "q_star"            : q_star, # The true unknown rewards
                 "tau"               : 0.5, # The tau parameter
                 "initialization"    : RandomInitialization, # Initialization strategy
                 "action_selection"  : softmax_action_selection, # Action selection strategy
                 "update_rule"       : update_rule # The update rule
                 }
    i_estimates_sm, i_total_reward_sm, i_mean_rewards_sm, i_regret_sm = bandit(bandit_inputs)
    rnd_estimates_sm += i_estimates_sm
    rnd_total_reward_sm += i_total_reward_sm
    rnd_mean_rewards_sm += i_mean_rewards_sm
    rnd_regret_sm += i_regret_sm

rnd_estimates_sm /= trials
rnd_total_reward_sm /= trials
rnd_mean_rewards_sm /= trials
rnd_regret_sm /= trials

print("True q values:{}".format(q_star))
plot_bandits(N, rnd_estimates_sm, rnd_total_reward_sm, rnd_mean_rewards_sm, rnd_regret_sm)

#### Compare all:

Let's see a comparison between the different initialization strategies for the softmax action selection strategy:

In [ ]:
comp_plot(regret_sm, min_regret_sm, avg_regret_sm, max_regret_sm, rnd_regret_sm, \
          mean_rewards_sm, min_mean_rewards_sm, avg_mean_rewards_sm, max_mean_rewards_sm, \
          rnd_mean_rewards_sm)

## Gradient bandits

## Exercise 1: Implement the gradient bandits action selection and update rule methods yourself:

#### Run the gradient bandits with zeros initialization:

In [ ]:
bandit_inputs = {"k"                 : k, # Number of coffee shops
                 "lr"                : 4., # Learning rate
                 "bias"              : True, # Bias (mean values)
                 "N"                 : N, # Number of trials
                 "k_t"               : np.zeros(k), # NOT NEEDED IN GRADIENT--- KEPT HERE TO HAVE THE SAME GENERAL ALGO
                 "q_star"            : q_star, # The true unknown rewards
                 "initialization"    : ZeroInitialization, # Initialization strategy
                 "action_selection"  : gradient_bandit_action_selection, # Action selection strategy
                 "update_rule"       : gradient_bandit_update_rule # The update rule
                }

estimates_grd, total_reward_grd, mean_rewards_grd, regret_grd = bandit(bandit_inputs)
print("True q values:{}".format(q_star))                 
plot_bandits(N, estimates_grd, total_reward_grd, mean_rewards_grd, regret_grd)

#### Will gradient bandits have a different result using other initializations?
Run the gradient bandits using different initializations to find out if you were correct:

## Comparing Bandit Algorithms
Compare the bandit algorithms through plotting.

In [ ]:
### Plot the results of all the algorithms ###
##############################################

fig, ax = plt.subplots()
ax.plot(np.linspace(0,N-1,N), np.cumsum(regret_rand), 'b-')
ax.plot(np.linspace(0,N-1,N), np.cumsum(regret_greedy), 'g-')
ax.plot(np.linspace(0,N-1,N), np.cumsum(regret_e_greedy), 'r-')
ax.plot(np.linspace(0,N-1,N), np.cumsum(regret_e_decay_greedy), 'c-')
ax.plot(np.linspace(0,N-1,N), np.cumsum(regret_ucb), 'm-')
ax.plot(np.linspace(0,N-1,N), np.cumsum(regret_sm), 'y-')
ax.set_xlabel("Time")
ax.set_ylabel("Speed")
ax.grid()
ax.set(xlabel='Time Steps', ylabel='Total Regret',
       title='Regret Curve')
plt.legend(['Random','Greedy','e-Greedy','e-Greedy with e decay','UCB','Softmax'])
plt.show()

fig, ax = plt.subplots()
ax.plot(np.linspace(0,N-1,N), mean_rewards_rand, 'b-')
ax.plot(np.linspace(0,N-1,N), mean_rewards_greedy, 'g-')
ax.plot(np.linspace(0,N-1,N), mean_rewards_e_greedy, 'r-')
ax.plot(np.linspace(0,N-1,N), mean_rewards_e_decay_greedy, 'c-')
ax.plot(np.linspace(0,N-1,N), mean_rewards_ucb, 'm-')
ax.plot(np.linspace(0,N-1,N), mean_rewards_sm, 'y-')
ax.set_xlabel("Time")
ax.set_ylabel("Speed")
ax.grid()
ax.set(xlabel='Time Steps', ylabel='Average Reward',
       title='Average Reward Curve')
plt.legend(['Random','Greedy','e-Greedy','e-Greedy with e decay','UCB','Softmax'])
plt.show()

## Exercise 2: Implement a bandit of your own liking!
(A hybrid bandit maybe? play with constant step size? compare different greedy ones? different gradient-based ones?)
